# Preliminary data analysis

In this notebook, we will carry out some preliminary data analysis for a single experimental run, to ensure that everything works properly, before writing a more general notebook which is able to analyse all of the experimental runs. The primary goal here is to make sure that we can carry out a finite-statistics analysis of the data and still obtain interesting results.

Read in the data from the associated xlsx file:

In [23]:
data = xlsread('MV0.9924_mi0.40_(29Agosto_1442_data1000)_parte1.xlsx');

**Extract the relevant data:**  
$n_{ax}$ : The raw counts for the events  
$\xi(a|x)$: The experimentally estimated probabilities  
$\mu$: the average number of photons in the pump pulse

In [32]:
nax = data(1:5,2:11)';
xiax = data(26:2:34,2:11)';
mu = data(12,2);

Given the data for $\mu$, define the states $\omega_x$ prepared in the experiment.  
We assume that at most 2 photons are produced, and hence truncate the Hilbert space up to 2 photons.  
We also assume that we are post-selecting on there being at least one photon detected

**Note**: I am assuming the old $\omega_4$ for the moment, since I don't know what the new state being produced is, however I don't believe this will change much in the following

In [33]:
p0 = 0; % we assume postselection
p1 = exp(-mu)*mu;
p2 = exp(-mu)*mu^2/2;

psum = p0 + p1 + p2;

p0 = p0/psum;
p1 = p1/psum;
p2 = p2/psum;

d = 15;
kets = eye(d);
omegax = zeros(d,d,5);

omegax(:,:,1) = p0*kets(:,1)*kets(1,:) + p1*kets(:,2)*kets(2,:) + p2*kets(:,6)*kets(6,:);
omegax(:,:,2) = p0*kets(:,1)*kets(1,:) + p1*kets(:,3)*kets(3,:) + p2*kets(:,7)*kets(7,:);
omegax(:,:,3) = p0*kets(:,1)*kets(1,:) + p1*kets(:,4)*kets(4,:) + p2*kets(:,8)*kets(8,:);
omegax(:,:,4) = p0*kets(:,1)*kets(1,:) + p1*kets(:,5)*kets(5,:) + p2*kets(:,9)*kets(9,:);

chi = 1/2*(1i*kets(:,2) + kets(:,3) -1i*kets(:,4) + kets(:,5));
chi2 = (-kets(:,6)+kets(:,7)-kets(:,8)+kets(:,9)+2*1i*kets(:,15)...
            +2*kets(:,14)+2*1i*kets(:,13)-2*1i*kets(:,12)+2*kets(:,11)-2*1i*kets(:,10));
chi2 = chi2/sqrt(trace(chi2*chi2'));

omegax(:,:,5) = p0*kets(:,1)*kets(1,:) + p1*chi*chi' + p2*chi2*chi2';

SDP to find the optimal dual variables $I_{ax}$ and $\nu$, which will give a bound of the form  
$$ P_g \leq \nu + \sum_{a,x}I_{ax}\xi(a|x) $$
valid for infinite statistics.  

The SDP solved is the following:
\begin{align}
P_g(x^*) = \max&\quad \mathrm{tr} \sum_{a} N_{aa} \omega_{x^*} \\
\text{s.t.}&\quad \xi(a|x) = \mathrm{tr}\sum_e N_{ae}\omega_x \quad \forall a,x \\
&\quad \sum_a N_{ae} = q(e) \mathbb{I} \quad \forall e \\
&\quad \sum_e q(e) = 1
\end{align}
where the maximisation is over POVMs $\mathbb{N} = \{N_{ae}\}_{ae}$ and probability distributions $\mathbf{q} = \{q(e)\}_e$. The dual variables of this SDP give rise to $I_{ax}$ and $\nu$

In [84]:
xs = 5;
[oa,ma] = size(xiax);

cvx_begin sdp quiet

    variable Nae(d,d,oa,oa) hermitian semidefinite
    variable qe(oa) nonnegative
    
    dual variables Iax{oa-1,ma}
    dual variables Xa{oa}
    dual variable nu
    
    maximise real(sumall(conj(repmat(omegax(:,:,xs),[1,1,oa,oa])).*Nae.*...
        permute(repmat(eye(oa),[1,1,d,d]),[3,4,1,2])))
    
    for a = 1:oa-1
        for x = 1:ma
            Iax{a,x} : xiax(a,x) == trace(sum(Nae(:,:,a,:),4)*omegax(:,:,x))
        end
    end
    
    
    for e = 1:oa
        Xa{e} : sum(Nae(:,:,:,e),3) == qe(e)*eye(d)
    end
    
    nu : sum(qe) == 1
    
cvx_end

Iax = cell2mat(Iax);

Pgfull = cvx_optval
Hminfull = -log2(Pgfull)

sumall(Iax.*(xiax(1:oa-1,:))) - nu;

Pgfull =
    0.4323
Hminfull =
    1.2100


SDP to find the guessing probability given finite statistics, using the Chernoff-Hoeffding inequality.  This fill find an inequality of the following form
$$ P_g \leq \nu + \sum_{ax} [u_{ax} - l_{ax}]\xi(a|x) + \sum_x t_x(\epsilon) \left[\sum_a (u_{ax} + l_{ax})\right] $$
where
$$ t_x(\epsilon) = \sqrt{\frac{\log(1/\epsilon)}{2\sum_a n_{ax}}} $$
and such that $u_{ax} \geq 0$ and $l_{ax} \geq 0$ for all $a,x$. 

The SDP solved is the following:
\begin{align}
P_g(x^*) = \max&\quad \mathrm{tr} \sum_{a} N_{aa} \omega_{x^*} \\
\text{s.t.}&\quad  \xi(a|x) - t_x(\epsilon) \leq \mathrm{tr}\sum_e N_{ae}\omega_x \leq \xi(a|x) + t_x(\epsilon) \quad \forall a,x\\
&\quad \sum_a N_{ae} = q(e) \mathbb{I} \quad \forall e \\
&\quad \sum_e q(e) = 1
\end{align}
where the maximisation is over POVMs $\mathbb{N} = \{N_{ae}\}_{ae}$, probability distributions $\mathbf{q} = \{q(e)\}_e$. The difference with the previous SDP is that we no longer require $\mathrm{tr}\sum_e N_{ae}\omega_x$ is equal to the observed probabilities (which we would only expect to be true in the limit of infinite statistics), but it should be bounded by the observed frequencies $\xi(a|x)$, depending on the number of counts $n_{ax}$ and a confidence parameter $\epsilon$, through the parameters $t_x(\epsilon)$.



In [88]:
epsi = 1E-9;

xs = 5;
[oa,ma] = size(xiax);

cvx_begin sdp quiet

    variable Nae(d,d,oa,oa) hermitian semidefinite
    variable qe(oa)
    variable Pax(oa,ma)
      
    maximise real(sumall(conj(repmat(omegax(:,:,xs),[1,1,oa,oa])).*Nae.*...
        permute(repmat(eye(oa),[1,1,d,d]),[3,4,1,2])))
    
    for a = 1:oa-1
        for x = 1:ma
            xiax(a,x) - sqrt(log(1/epsi)/2/sum(nax(:,x))) <= trace(sum(Nae(:,:,a,:),4)*omegax(:,:,x))
            trace(sum(Nae(:,:,a,:),4)*omegax(:,:,x)) <= xiax(a,x) + sqrt(log(1/epsi)/2/sum(nax(:,x))) 
        end
    end
    
    
    for e = 1:oa 
        sum(Nae(:,:,:,e),3) == qe(e)*eye(d)
    end
    
    sum(qe) == 1
    
cvx_end

Pgfull = cvx_optval
Hminfull = -log2(Pgfull)


Pgfull =
    0.5129
Hminfull =
    0.9632


Thus, due to the finite statistics, we see that we obtain have a decrease in min entropy of 0.2468 due to the finite statistics, dropping us just below 1 bit/round